## Nimeüksused lausetes test5

In [22]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from collections import Counter
import itertools
import csv
from copy import deepcopy

Tühi järjend lausete jaoks:

In [2]:
corpus = []

Tühi järjend lausete ID-de jaoks:

In [3]:
sentence_ids = []

Andmebaasist loetakse sisse laused:

In [4]:
con = sqlite3.connect("media_data.db")
cur = con.cursor()

In [5]:
for row in cur.execute("SELECT ID, sentence FROM sentences WHERE named_entity = 1"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    corpus.append(sentence)
    sentence_ids.append(sentence_id)

In [6]:
con.close()

Vaatame üht lauset ja selle nimeüksusi:

In [7]:
for ner in corpus[5].ner:
    print(ner.nertag, ner.text, ner.enclosing_text)

LOC ['Eestis'] Eestis
PER ['Eerik'] Eerik


NerTagger võib olla teinud vigu, näiteks märkinud isikunime asukohaks, asukoha organisatsiooniks jne. Siin katsetan lihtsakoelist lahendust, mis ei korrigeeri kindlasti kõiki selliseid vigu, kuid võib nende vigade hulka vähendada. Nimelt loetakse kokku eri nimeüksuste esinemised kõigis liikides. Kui nimeüksus esineb rohkem kui ühes liigis, loetakse õigeks liigiks see, milles tema esinemissagedus kõige suurem on. Kui esinemissagedused on eri liikides võrdsed, ei tehta aga praeguses etapis midagi.

Esiteks loon tühjad järjendid nii isikunimede, organisatsiooninimede kui asukohanimede jaoks:

In [8]:
entities_PER = []
entities_ORG = []
entities_LOC = []

Seejärel käin läbi kõik korpuses olevad laused ja lisan nimeüksused järjenditesse vastavalt nende liigile:

In [9]:
for sentence in corpus:
    for ner in sentence.ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        if ner.nertag == 'PER':
            entities_PER.append(' '.join(lemmatized_ner))
        elif ner.nertag == 'ORG':
            entities_ORG.append(' '.join(lemmatized_ner))
        else:
            entities_LOC.append(' '.join(lemmatized_ner))

In [10]:
len(entities_PER)

320

In [11]:
len(entities_ORG)

152

In [12]:
len(entities_LOC)

353

Seejärel tuleb tekitada kõigi nimeolemiliikide sagedusloendid:

In [13]:
entities_PER_lower = [name.lower() for name in entities_PER]
entities_ORG_lower = [name.lower() for name in entities_ORG]
entities_LOC_lower = [name.lower() for name in entities_LOC]

In [14]:
frequencies_PER = Counter(entities_PER_lower)
frequencies_ORG = Counter(entities_ORG_lower)
frequencies_LOC = Counter(entities_LOC_lower)

In [15]:
#Kümme sagedasemat isiku-, organisatsiooni- ja asukohanime
print('PER' + '\t' + 'ORG' + '\t' + 'LOC')
for (per, org, loc) in itertools.zip_longest(frequencies_PER.most_common(10), frequencies_ORG.most_common(10), frequencies_LOC.most_common(10)):
    print(per, org, loc)

PER	ORG	LOC
('edgar savisaar', 17) ('keskerakond', 20) ('eesti', 98)
('jüri', 10) ('rein', 8) ('tallinn', 45)
('andrus ansip', 8) ('irl', 7) ('euroopa', 18)
('ansip', 8) ('euroopa liit', 5) ('hiina', 15)
('kaarel', 8) ('reformierakond', 5) ('tartu', 10)
('rein', 7) ('euroopa liidu', 5) ('soome', 8)
('andrus', 7) ('kapo', 4) ('rootsi', 4)
('marek strandberg', 6) ('riigikogu', 4) ('läti', 4)
('allar jõks', 5) ('eesti energia', 3) ('euroopa liit', 4)
('strandberg', 5) ('bns', 3) ('eesti vabariik', 3)


Loon uued tühjad järjendid isiku-, organisatsiooni- ja asukohanimede jaoks, mida võib kasutada nimede mugavaks andmebaasi lisamiseks.

In [16]:
entities_PER_new = []
entities_ORG_new = []
entities_LOC_new = []

In [17]:
frequencies_PER_copy = deepcopy(frequencies_PER)
frequencies_ORG_copy = deepcopy(frequencies_ORG)
frequencies_LOC_copy = deepcopy(frequencies_LOC)

In [18]:
equal_occurrences = []

In [19]:
#Kas isikunimede hulgas olevad nimed leiduvad ka organisatsiooni- või asukohanimede hulgas:
for name in frequencies_PER_copy.keys():
    if frequencies_ORG_copy[name] > 0:
        if frequencies_ORG_copy[name] > frequencies_PER_copy[name]:
            print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_ORG_copy[name] < frequencies_PER_copy[name]:
            print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_ORG[name]
        else:
            equal_occurrences.append(name)
                
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_PER_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_LOC_copy[name] < frequencies_PER_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Kas organisatsiooninimede hulgas leidub nimesid, mis esinevad ka asukohanimede hulgas (isikunimesid pole vaja uuesti läbi käia):
for name in frequencies_ORG_copy.keys():
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_ORG_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_ORG[name]
        elif frequencies_LOC_copy[name] < frequencies_ORG_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Asukohanimesid pole vaja eraldi läbi käia, seal leiduvad duplikaadid kas isiku- või organisatsiooninimedest on juba tuvastatud.  

rein ORG: 8 PER: 7
euroopa liit LOC: 4 ORG: 5


In [20]:
print(equal_occurrences)

['teder', 'ekre', 'taska', 'levadia']


In [21]:
for name in frequencies_PER.keys():
    entities_PER_new.append(name)
    
for name in frequencies_ORG.keys():
    entities_ORG_new.append(name)
    
for name in frequencies_LOC.keys():
    entities_LOC_new.append(name)

Saadud parandustega sagedusloendid kirjutatakse talletatakse csv-failis, et võimaldada vajadusel käsitsi korrektuuride tegemist.

In [28]:
with open('ner_frequencies.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['name', 'freq', 'value'])
    writer.writeheader()
    header2 = {'name': 'string', 'freq': 'integer', 'value': 'string'}
    writer.writerow(header2)
    for name in frequencies_PER.keys():
        writer.writerow({'name': name, 'freq': frequencies_PER[name], 'value': 'PER'})
    for name in frequencies_ORG.keys():
        writer.writerow({'name': name, 'freq': frequencies_ORG[name], 'value': 'ORG'})
    for name in frequencies_LOC.keys():
        writer.writerow({'name': name, 'freq': frequencies_LOC[name], 'value': 'LOC'})

In [29]:
with open('ner_frequencies.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [30]:
print(rows)

[{'name': 'string', 'freq': 'integer', 'value': 'string'}, {'name': 'hans', 'freq': '4', 'value': 'PER'}, {'name': 'eerik', 'freq': '1', 'value': 'PER'}, {'name': 'andrus ansip', 'freq': '8', 'value': 'PER'}, {'name': 'meelis', 'freq': '2', 'value': 'PER'}, {'name': 'olev esula', 'freq': '1', 'value': 'PER'}, {'name': 'vadim', 'freq': '2', 'value': 'PER'}, {'name': 'juskin', 'freq': '1', 'value': 'PER'}, {'name': 'margus', 'freq': '2', 'value': 'PER'}, {'name': 'sven', 'freq': '4', 'value': 'PER'}, {'name': 'siim kallas', 'freq': '1', 'value': 'PER'}, {'name': 'deniss borodits', 'freq': '1', 'value': 'PER'}, {'name': 'jukuminek', 'freq': '1', 'value': 'PER'}, {'name': 'borodits', 'freq': '1', 'value': 'PER'}, {'name': 'jüri', 'freq': '10', 'value': 'PER'}, {'name': 'eiki', 'freq': '2', 'value': 'PER'}, {'name': 'kerviel', 'freq': '1', 'value': 'PER'}, {'name': 'andres', 'freq': '4', 'value': 'PER'}, {'name': 'toomas hendrik', 'freq': '4', 'value': 'PER'}, {'name': 'meri', 'freq': '2', 

Järgnevalt luuakse andmebaasi uus tabel huvipakkuvate nimeüksuste hoiustamiseks (isiku- ja organisatsiooninimed).

In [179]:
con = sqlite3.connect("media_data.db")
cur = con.cursor()

In [181]:
cur.execute("CREATE TABLE named_entities(ID INTEGER PRIMARY KEY, sentence_ID INTEGER, PER TEXT, ORG TEXT)")

In [182]:
for i in range(len(corpus)):
    for ner in corpus[i].ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        lemmatized_ner_text = ' '.join(lemmatized_ner).lower()
        if ner.nertag == 'PER' and lemmatized_ner_text in entities_PER_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'PER' and lemmatized_ner_text in entities_PER_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag == 'ORG' and lemmatized_ner_text in entities_ORG_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'ORG' and lemmatized_ner_text in entities_ORG_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        con.commit()

In [183]:
con.close()

Praeguse seisuga ei püüta parandada vigaseks hinnatud ner-olemeid, vaid luua ainult parandatud kujul nimekiri ner-olemitest